In [1]:
import pandas as pd
import json
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import base64
from io import BytesIO
from sklearn.metrics import confusion_matrix
import os

df1 = pd.read_excel("datasets/result_hack1.xlsx")
df2 = pd.read_excel("datasets/result_hack2.xlsx")
df = pd.concat([df1, df2], ignore_index=True)

df_orig = df.copy()

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332204 entries, 0 to 1332203
Data columns (total 17 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   FLIGHT KEY            1331880 non-null  object        
 1   PASSENGERS            1332204 non-null  int64         
 2   NOMBRE DE AEROLINEA   1332204 non-null  object        
 3   FECHA                 1332204 non-null  datetime64[ns]
 4   ORIGEN                1330588 non-null  object        
 5   DESTINO               1330588 non-null  object        
 6   FLIGHT NO             1331880 non-null  float64       
 7   DEPARTUTE LOCAL TIME  1320414 non-null  datetime64[ns]
 8   ARRIVAL LOCAL TIME    1332204 non-null  datetime64[ns]
 9   SALES                 1313790 non-null  float64       
 10  TYPE TRANSACTION      1313790 non-null  object        
 11  CATEGORY              1313732 non-null  object        
 12  SUPERCATEGORY         1313714 non-null  ob

In [2]:
# PREPROCESSING

# Remove entries with null values (oversimplification of a real process)
df = df.dropna()

df['ITEM CODE'] = df['ITEM CODE'].astype('int64')
df['SALES'] = df['SALES'].astype('int64')

# In this case, i'm only using SALE type transactions
df = df[df['TYPE TRANSACTION'] == 'SALE'].reset_index(drop=True)

# Group entries
agg_dict = {col: 'first' for col in df.columns if col not in ['SALES', 'LOST SALES']}
agg_dict.update({'SALES': 'sum', 'LOST SALES': 'sum'})

grouped = (
    df.groupby(['FLIGHT KEY', 'CATEGORY'], as_index=False)
      .agg(agg_dict)
)

df = grouped

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401214 entries, 0 to 401213
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   FLIGHT KEY            401214 non-null  object        
 1   PASSENGERS            401214 non-null  int64         
 2   NOMBRE DE AEROLINEA   401214 non-null  object        
 3   FECHA                 401214 non-null  datetime64[ns]
 4   ORIGEN                401214 non-null  object        
 5   DESTINO               401214 non-null  object        
 6   FLIGHT NO             401214 non-null  float64       
 7   DEPARTUTE LOCAL TIME  401214 non-null  datetime64[ns]
 8   ARRIVAL LOCAL TIME    401214 non-null  datetime64[ns]
 9   TYPE TRANSACTION      401214 non-null  object        
 10  CATEGORY              401214 non-null  object        
 11  SUPERCATEGORY         401214 non-null  object        
 12  ITEM CODE             401214 non-null  int64         
 13 

In [3]:
# Convert date-time columns
df['MONTH'] = df['FECHA'].dt.month
df['DAY_OF_WEEK'] = df['FECHA'].dt.weekday 

# Create cyclic variables
df['month_sin'] = np.sin(2 * np.pi * df['MONTH']/12)
df['month_cos'] = np.cos(2 * np.pi * df['MONTH']/12)
df['dow_sin'] = np.sin(2 * np.pi * df['DAY_OF_WEEK']/7)
df['dow_cos'] = np.cos(2 * np.pi * df['DAY_OF_WEEK']/7)

df['DURATION_MIN'] = (df['ARRIVAL LOCAL TIME'] - df['DEPARTUTE LOCAL TIME']).dt.total_seconds() / 60 

# Extracting hour
df['HOUR'] = df['DEPARTUTE LOCAL TIME'].dt.hour

def hour_slot(hour):
    if 0 <= hour <= 5:
        return 'EarlyMorning'
    elif 6 <= hour <= 11:
        return 'Morning'
    elif 12 <= hour <= 13:
        return 'Noon'
    elif 14 <= hour <= 17:
        return 'Afternoon'
    elif 18 <= hour <= 21:
        return 'Evening'
    else:
        return 'LateNight'

df['HOUR_SLOT'] = df['HOUR'].apply(hour_slot)
# One-hot encoding
df = pd.get_dummies(df, columns=['HOUR_SLOT'], prefix='hour', dtype=int)

# Drop original datetime columns
drop_cols = ['FECHA', 'DEPARTUTE LOCAL TIME', 'ARRIVAL LOCAL TIME', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'TYPE TRANSACTION']
df = df.drop(columns=drop_cols)

######################################
# In order to use properly the origin and destination info, we obtain the mean consuming of that city
# Obtain total sales per flight
flight_sales = df.groupby('FLIGHT KEY')['SALES'].sum().reset_index().rename(columns={'SALES': 'TOTAL_SALES'})

# Calculate global mean
df = df.merge(flight_sales, on='FLIGHT KEY', how='left')
global_mean = df['TOTAL_SALES'].mean()

# Target encoding function
def target_encode_smooth(df, col, target, m=5):
    """
    df: DataFrame
    col: column to modify
    target: target column
    m: smoothing factor
    """
    agg = df.groupby(col)[target].agg(['mean', 'count'])
    # smoothing
    agg['encoded'] = (agg['mean'] * agg['count'] + global_mean * m) / (agg['count'] + m)
    return agg['encoded']

origin_encoding = target_encode_smooth(df, 'ORIGEN', 'TOTAL_SALES', m=5)
dest_encoding   = target_encode_smooth(df, 'DESTINO', 'TOTAL_SALES', m=5)

## Save for later use
origin_dict = origin_encoding.to_dict()
dest_dict = dest_encoding.to_dict()

# Guardar en un JSON
encoding_dict = {
    "origin": origin_dict,
    "destination": dest_dict
}

with open("mappers/city_mean_sales.json", "w") as f:
    json.dump(encoding_dict, f, indent=4)


df['MEAN_ORIGIN_CONSUPTION'] = df['ORIGEN'].map(origin_encoding)
df['MEAN_DEST_CONSUPTION'] = df['DESTINO'].map(dest_encoding)

df = df.drop(columns=['ORIGEN', 'DESTINO', 'TOTAL_SALES', 'WAREHOUSE',])
######################################

# Convert categorical variables into binary
categorical_cols = ['CATEGORY','SUPERCATEGORY', 'NOMBRE DE AEROLINEA', 'CURRENCY']
df = pd.get_dummies(df, columns=categorical_cols, prefix=[c.lower() for c in categorical_cols], dtype=int)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401214 entries, 0 to 401213
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   FLIGHT KEY                              401214 non-null  object 
 1   PASSENGERS                              401214 non-null  int64  
 2   FLIGHT NO                               401214 non-null  float64
 3   ITEM CODE                               401214 non-null  int64  
 4   SALES                                   401214 non-null  int64  
 5   LOST SALES                              401214 non-null  int64  
 6   month_sin                               401214 non-null  float64
 7   month_cos                               401214 non-null  float64
 8   dow_sin                                 401214 non-null  float64
 9   dow_cos                                 401214 non-null  float64
 10  DURATION_MIN                            4012

In [4]:
# Save product code
df_ids = df[['ITEM CODE']].copy

In [5]:
# Separate target and independient variables
X = df.drop(columns=['FLIGHT KEY', 'FLIGHT NO'])
X.to_csv('models/data.csv',index=False)
X = X.drop(columns=['ITEM CODE', 'SALES'])
y = df['SALES']

In [6]:
# Model Training
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Training split
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.7, shuffle=True, random_state=42)

numerical_columns = ['PASSENGERS','DURATION_MIN','MEAN_ORIGIN_CONSUPTION','MEAN_DEST_CONSUPTION', 'month_sin', 'month_cos', 'dow_sin', 'dow_cos', 'LOST SALES']

# Scale the data
scaler = StandardScaler()
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
X_test[numerical_columns] = scaler.fit_transform(X_test[numerical_columns])

X_train.head(10)

,PASSENGERS,LOST SALES,month_sin,month_cos,dow_sin,dow_cos,DURATION_MIN,hour_Afternoon,hour_EarlyMorning,hour_Evening,...,category_Savoury Snacks,category_Skincare & Make-up,category_Sweet Snacks,category_Tobacco,category_Tobacco.,supercategory_BISTRO,supercategory_BOUTIQUE,supercategory_DUTY FREE,nombre de aerolinea_GateGroup Airlines,currency_EUR
170883,-1.054679,-0.072747,0.934014,-0.264699,-0.583000,-1.242938,-0.316044,0,0,0,...,0,0,0,0,0,1,0,0,1,1
154772,-0.335321,-0.072747,0.368652,-0.841812,-1.350738,-0.285794,-0.343533,0,0,0,...,0,0,0,0,0,1,0,0,1,1
130296,-0.452768,0.407183,0.368652,1.889114,0.032677,1.438919,-0.144241,1,0,0,...,0,0,0,0,0,1,0,0,1,1
191633,0.560206,-0.072747,0.368652,-0.841812,0.032677,1.438919,-0.235870,1,0,0,...,0,0,0,0,0,1,0,0,1,1
129776,-0.731702,-0.072747,-0.403646,-1.053050,-1.350738,-0.285794,-0.174021,0,0,0,...,0,0,1,0,0,1,0,0,1,1
83244,-0.790425,-0.072747,-1.741306,-0.264699,1.416091,-0.285794,-0.350405,0,0,0,...,0,0,0,0,0,1,0,0,1,1
23973,0.251909,-0.072747,-1.175945,-0.841812,-1.350738,-0.285794,-0.329789,0,0,0,...,0,0,0,0,0,1,0,0,1,1
167761,-0.966595,-0.072747,-1.741306,-0.264699,0.648354,-1.242938,-0.249614,0,0,1,...,1,0,0,0,0,1,0,0,1,1
329074,-1.348295,-0.072747,0.934014,-0.264699,-1.076735,0.907745,2.845129,0,0,1,...,0,0,0,0,0,1,0,0,1,1
128142,0.295952,-0.072747,0.934014,-0.264699,-0.583000,-1.242938,-0.153404,0,0,0,...,1,0,0,0,0,1,0,0,1,1


In [7]:
mlp = MLPRegressor(hidden_layer_sizes=(69,), validation_fraction=0.15, max_iter=1000)
mlp.fit(X_train, y_train)

,loss,'squared_error'
,hidden_layer_sizes,"(69,)"
,activation,'relu'
,solver,'adam'
,alpha,0.0001
,batch_size,'auto'
,learning_rate,'constant'
,learning_rate_init,0.001
,power_t,0.5
,max_iter,1000
,shuffle,True


In [8]:
# Evaluate the model
from sklearn.metrics import mean_squared_error, mean_absolute_error

prediction = mlp.predict(X_test)
score = mlp.score(X_test, y_test)
mse = mean_squared_error(y_test, prediction)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, prediction)

print(f"Score: {score}")
print(f"Mean Square Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Square Error: {rmse}")

Score: 0.5039196850977685
Mean Square Error: 7.44267010797601
Mean Absolute Error: 1.7086359780375222
Root Mean Square Error: 2.7281257500298643


In [9]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [10]:
prediction = rf.predict(X_test)

# Evaluation
score2 = rf.score(X_test, y_test)
mse2 = mean_squared_error(y_test, prediction)
rmse2 = np.sqrt(mse2)
mae2 = mean_absolute_error(y_test, prediction)

# Results
print(f"R Score: {score2:.4f}")
print(f"Mean Square Error: {mse2:.4f}")
print(f"Root Mean Square Error: {rmse2:.4f}")
print(f"Mean Absolute Error: {mae2:.4f}")

R Score: 0.5210
Mean Square Error: 7.1866
Root Mean Square Error: 2.6808
Mean Absolute Error: 1.7001


In [11]:
# Gradient Boosting
from lightgbm import LGBMRegressor
gb = LGBMRegressor(random_state=42)
gb.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003155 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 881
[LightGBM] [Info] Number of data points in the train set: 280849, number of used features: 33
[LightGBM] [Info] Start training from score 3.833829


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [12]:
prediction = gb.predict(X_test)

score3 = gb.score(X_test, y_test)
mse3 = mean_squared_error(y_test, prediction)
rmse3 = np.sqrt(mse3)
mae3 = mean_absolute_error(y_test, prediction)

print(f"Score: {score3}")
print(f"Mean Square Error: {mse3}")
print(f"Mean Absolute Error: {mae3}")
print(f"Root Mean Square Error: {rmse3}")

Score: 0.5335705191404234
Mean Square Error: 6.997820010972429
Mean Absolute Error: 1.6852975566567607
Root Mean Square Error: 2.645339299782247


In [13]:
# Improved Gradient Boosting Regressor
from sklearn.model_selection import RandomizedSearchCV

lgbm = LGBMRegressor(random_state=42, verbose=-1)

# DDefining a rangge of hyperparameters
param_grid = {
    'n_estimators': [500, 800, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [8, 12, 15],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5]
}

search = RandomizedSearchCV(
    estimator=lgbm,
    param_distributions=param_grid,
    n_iter=20,           
    scoring='r2',
    cv=3,               
    random_state=42,
)

search.fit(X_train, y_train)

# Best model
best_model = search.best_estimator_
print("Best params:", search.best_params_)

Best params: {'subsample': 0.7, 'reg_lambda': 0.1, 'reg_alpha': 0.5, 'n_estimators': 1000, 'min_child_samples': 20, 'max_depth': 12, 'learning_rate': 0.1, 'colsample_bytree': 1.0}


In [14]:
predictions = best_model.predict(X_test)
score4 = best_model.score(X_test, y_test)
mse4 = mean_squared_error(y_test, prediction)
rmse4 = np.sqrt(mse3)
mae4 = mean_absolute_error(y_test, prediction)

print(f"Score: {score4}")
print(f"Mean Square Error: {mse4}")
print(f"Mean Absolute Error: {mae4}")
print(f"Root Mean Square Error: {rmse4}")

Score: 0.5657472064744573
Mean Square Error: 6.997820010972429
Mean Absolute Error: 1.6852975566567607
Root Mean Square Error: 2.645339299782247


In [15]:
# Save the model and the scaler
joblib.dump(best_model, 'models/model.pkl')
joblib.dump(scaler, 'models/scaler.pkl')

['models/scaler.pkl']